#### Mapping Declaratively with Reflected Tables

There are _several patterns available_ which provide for producing _mapped classes_ against a _series_ of `Table` objects that were __introspected from the database__, using the _reflection process_ described at `Reflecting Database Objects`.

A _very simple way_ to __map a class to a table reflected from the database__ is to use a `declarative hybrid mapping`, passing the `Table.autoload_with` parameter to the `Table`.

In [1]:
from sqlalchemy import Column, Integer, String, Table, ForeignKey, create_engine
from sqlalchemy.orm import relationship, declarative_base
from sqlalchemy.ext.declarative import DeferredReflection

In [2]:
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)
Base = declarative_base()

In [3]:
class UserInline(Base):
    __table__ = Table(
        "user",
        Base.metadata,
        Column("id", Integer, primary_key=True),
        Column("name", String),
        Column("fullname", String),
        Column("nickname", String),
    )

In [4]:
Base.metadata.create_all(engine)

2022-11-01 16:15:10,478 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-01 16:15:10,481 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user")
2022-11-01 16:15:10,482 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-01 16:15:10,485 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("user")
2022-11-01 16:15:10,486 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-01 16:15:10,488 INFO sqlalchemy.engine.Engine 
CREATE TABLE user (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


2022-11-01 16:15:10,490 INFO sqlalchemy.engine.Engine [no key 0.00128s] ()
2022-11-01 16:15:10,492 INFO sqlalchemy.engine.Engine COMMIT


In [5]:
class User(Base):
    __table__ = Table(
        "user",
        Base.metadata,
        autoload_with=engine,
    )

A _variant_ on the above pattern that __scales much better__ is to use the `MetaData.reflect()` method to __reflect a full set of `Table` objects at once__, then refer to them from the `MetaData`.

In [6]:
Base.metadata.reflect(engine)

2022-11-01 16:15:10,644 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-01 16:15:10,647 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2022-11-01 16:15:10,649 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-01 16:15:10,650 INFO sqlalchemy.engine.Engine ROLLBACK


In [7]:
class ReflectedUser(Base):
    __table__ = Base.metadata.tables["user"]

A __major downside__ to the above approach is that the _mapped classes cannot be declared until the tables have been reflected_, which __requires__ the `database connectivity source` __to be present__ while the `application classes` are _being declared_; it's _typical_ that _classes are declared as the modules of an application are being imported_, but `database connectivity` __isn't available until the application starts__ running code so that it can _consume configuration information_ and __create an engine__. There are _currently_ `two approaches` to __working around__ this.

##### Using DeferredReflection

To _accommodate_ the use case of _declaring mapped classes_ where `reflection of table metadata` can __occur afterwards__, a _simple extension_ called the `DeferredReflection mixin` is available, which __alters__ the _declarative mapping process_ to be __delayed until__ a special class-level `DeferredReflection.prepare()` method is called, which will __perform the reflection process__ against a `target database`, and will _integrate the results_ with the `declarative table mapping process`, that is, classes which use the `__tablename__` attribute.

In [8]:
class FooBase(Base):
    __table__ = Table(
        "foo",
        Base.metadata,
        Column("id", Integer, primary_key=True),
        Column("name", String),
        Column("fullname", String),
        Column("nickname", String),
    )

In [9]:
class BarBase(Base):
    __table__ = Table(
        "bar",
        Base.metadata,
        Column("id", Integer, primary_key=True),
        Column("foo_id", ForeignKey("foo.id")),
    )

In [10]:
Base.metadata.create_all(engine)

2022-11-01 16:15:10,926 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-01 16:15:10,928 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("user")
2022-11-01 16:15:10,929 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-01 16:15:10,933 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("foo")
2022-11-01 16:15:10,935 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-01 16:15:10,936 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("foo")
2022-11-01 16:15:10,937 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-01 16:15:10,939 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("bar")
2022-11-01 16:15:10,940 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-01 16:15:10,941 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("bar")
2022-11-01 16:15:10,942 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-01 16:15:10,944 INFO sqlalchemy.engine.Engine 
CREATE TABLE foo (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


In [11]:
class Reflected(DeferredReflection):
    __abstract__ = True

In [12]:
class Foo(Reflected, Base):
    __tablename__ = "foo"

In [13]:
class Bar(Reflected, Base):
    __tablename__ = "bar"

Above, we create a _mixin class_ `Reflected` that will __serve as a base__ for classes in our _declarative hierarchy_ that should become mapped when the `Reflected.prepare` method is called. The above mapping is __not complete until we do so__, given an `Engine`.

In [14]:
Reflected.prepare(engine)

2022-11-01 16:15:11,426 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-11-01 16:15:11,428 INFO sqlalchemy.engine.Engine PRAGMA main.table_xinfo("foo")
2022-11-01 16:15:11,431 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-01 16:15:11,433 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-11-01 16:15:11,434 INFO sqlalchemy.engine.Engine [raw sql] ('foo',)
2022-11-01 16:15:11,436 INFO sqlalchemy.engine.Engine PRAGMA main.foreign_key_list("foo")
2022-11-01 16:15:11,437 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-01 16:15:11,438 INFO sqlalchemy.engine.Engine PRAGMA temp.foreign_key_list("foo")
2022-11-01 16:15:11,439 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-11-01 16:15:11,440 INFO sqlalchemy.engine.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = ? AND type = 'table'
2022-11-01 16:15:11,441 INFO s

The _purpose_ of the `Reflected` class is to __define the scope at which classes should be reflectively mapped__. The plugin will _search_ among the `subclass tree` of the target against which `.prepare()` is called and __reflect all tables which are named by declared classes__; tables in the _target database_ that are _not part of mappings_ and are _not related to the target tables via foreign key constraint_ __will not be reflected__.

##### Using Automap

A more _automated solution_ to __mapping against an existing database__ where _table reflection_ is to be used is to use the `Automap` extension. This extension will __generate entire mapped classes from a database schema__, __including relationships between classes__ based on _observed_ `foreign key` _constraints_. While it _includes hooks for customization_, such as hooks that allow __custom class naming__ and __relationship naming__ schemes, `automap` is oriented towards an expedient __zero-configuration style of working__. If an application wishes to have a _fully explicit model_ that makes use of _table reflection_, the `Using DeferredReflection` may be _preferable_.